<a href="https://colab.research.google.com/github/pbelevich/csprng/blob/encrypt_decrypt_notebook/notebooks/encrypt_decrypt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyTorch/CSPRNG encrypt/decrypt examples

torchcsprng 0.2.0 exposes new API for tensor encryption/decryption. Tensor encryption/decryption API is dtype agnostic, so a tensor of any dtype can be encrypted and the result can be stored to a tensor of any dtype. An encryption key also can be a tensor of any dtype. Currently torchcsprng supports AES cipher with 128-bit key in two modes: ECB and CTR.

In [1]:
# !pip install --pre torchcsprng -f https://download.pytorch.org/whl/nightly/cpu/torch_nightly.html
# for CUDA runtime:
!pip install --pre torchcsprng -f https://download.pytorch.org/whl/nightly/cu101/torch_nightly.html

Looking in links: https://download.pytorch.org/whl/nightly/cu101/torch_nightly.html
     |████████████████████████████████| 4.4MB 552kB/s 
     |████████████████████████████████| 788.6MB 25kB/s 
ERROR: torchvision 0.8.1+cu101 has requirement torch==1.7.0, but you'll have torch 1.8.0.dev20201210+cu101 which is incompatible.
  Found existing installation: torch 1.7.0+cu101
    Uninstalling torch-1.7.0+cu101:
      Successfully uninstalled torch-1.7.0+cu101


In [2]:
import torch
import torchcsprng as csprng

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

torchcsprng implementation of AES with 128 bit key requires to have a key tensor of 16 bytes but of any dtype

In [4]:
key = torch.empty(16, dtype=torch.uint8, device=device).random_(0, 256)
key

tensor([ 94, 155, 111,  73,  36,  65,  95, 175, 103, 249,  39, 221, 225, 106,
        147,  18], dtype=torch.uint8)

Alternatively it can be a tensor of 8 elements of `torch.int16` or even 4 elements of `torch.float32`

The size of input tensor is 42 * (32/8) = 168 bytes. AES 128 operates with 16-bytes blocks, so zero-padding of 8 bytes will be used to form 176 bytes(eleven 16-bytes blocks)

In [5]:
initial = torch.empty(42, dtype=torch.float32, device=device).normal_(-24.0, 42.0)
initial

tensor([ -33.2154,   -4.4617,   -2.9105,  -37.1893,   -6.0623,  -47.4273,
         -44.7682,   23.7199,   52.3595,  -80.1355,   10.7041,  -52.0069,
         -59.1335,  -51.7051,  -96.7531,  -47.6263,  -37.4668,  -26.9140,
          -8.7239,  -68.0884,  -73.8865,  -35.0378,  -24.9022,  -71.3780,
         -83.3700,  -73.9181,  -56.9136,  -14.0332,   -0.1193,  -24.6635,
         -12.3755,   26.1501,  -21.7292,   61.0362,   -8.2424,  -27.3287,
        -116.2249,  -32.5120,  -56.9908,  -44.0908,   15.9910,  -12.2838])

torchcsprng requires output tensor to be of the same size in bytes as input tensor rounded up to 16 bytes(AES 128 block size), so if `torch.int64` is dtype of the destination tensor size must be 176 / (64/8) = 22

In [6]:
encrypted = torch.empty(22, dtype=torch.int64, device=device)

Call `torchcsprng.encrypt` to encrypt `initial` tensor in [ECB](https://en.wikipedia.org/wiki/Block_cipher_mode_of_operation#Electronic_codebook_(ECB)) mode with 128-bit `key` tensor and store the result to `encrypted` tensor.

In [7]:
csprng.encrypt(initial, encrypted, key, "aes128", "ecb")

tensor([-1712310488599806515, -8311769289706173237, -8219472943074007291,
         4944862344658876406,  5289614356287313567, -6556419268017486570,
        -2070634943111059129, -6988005632242749066, -6786984565899259673,
         2617853683610034142,  7394987086142621793,  9037048344238914391,
        -4389724417591829293,   639143755047053321,  6630155308790632093,
         8233678956273288167,  7762736707078557167,   -66820074992426839,
         2506358826618882964,  3811127519115226076,  7223286519671228245,
        -4680879586720463643])

Create an output tensor

In [8]:
decrypted = torch.empty_like(initial)

Call `torchcsprng.decrypt` to decrypt `encrypted` tensor in ECB mode with 128-bit `key` tensor and store the result to `decrypted` tensor.

In [10]:
# csprng.decrypt(encrypted, decrypted, key, "aes128", "ecb")